In [6]:
import numpy as np


class GridWorld:
    def __init__(self, size=4):
        self.size = size
        self.nS = size * size
        self.nA = 4
        self.P = self._build_transition_probabilities()

    def _build_transition_probabilities(self):
        P = {}
        for s in range(self.nS):
            P[s] = {a: [] for a in range(self.nA)}

        for row in range(self.size):
            for col in range(self.size):
                s = row * self.size + col
                for a in range(self.nA):
                    li = P[s][a]
                    if a == 0:
                        li.append((1.0, self._get_state(s, row, col - 1), -1.0, False))
                    elif a == 1:
                        li.append((1.0, self._get_state(s, row - 1, col), -1.0, False))
                    elif a == 2:
                        li.append((1.0, self._get_state(s, row, col + 1), -1.0, False))
                    elif a == 3:
                        li.append((1.0, self._get_state(s, row + 1, col), -1.0, False))
                    else:
                        raise ValueError("Invalid action: {}".format(a))
                    
        # Terminal states
        for a in range(self.nA):
            P[0][a] = [(1.0, 0, 0.0, True)]
            P[15][a] = [(1.0, 15, 0.0, True)]

        return P
    
    def _get_state(self, s, row, col):
        if row < 0 or row >= self.size or col < 0 or col >= self.size:
            return s
        return row * self.size + col


def iterative_policy_evaluation(policy, env, discount_factor=1.0, theta=0.00001, max_iterations=1000):
    """
    Iterative policy evaluation algorithm.
    Evaluates the value function for a given policy.

    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment.
            env.nA is a number of actions in the environment.
        discount_factor: Lambda discount factor.
        theta: We stop evaluation once our value function change is less than theta for all states.

    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V_old = np.zeros(env.nS)
    for i in range(max_iterations):
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value.
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v

        # Sutton and Barto's example 4.1 uses a backup copy of the value function
        # Updating the values in-place will result in different intermediate values,
        # but should still converge to the same answer.
        # V_old = V.copy()

        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)


# Create an environment based on the 4x4 gridworld
env = GridWorld()

# Create a random policy
random_policy = np.ones([env.nS, env.nA]) / env.nA

# Evaluate the value function
v = iterative_policy_evaluation(random_policy, env, discount_factor=1.0, theta=0.00001, max_iterations=1000)

# Print the value function
print("Reshaped grid value function (rounded):")
print(np.round(v.reshape(env.size, env.size), decimals=1))


Reshaped grid value function (rounded):
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
